In [1]:
#! python3 -m pip install --upgrade pip
#! pip install mplfinance
from xdata_config import *
from functions_module import *
import mplfinance as mpf
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model
from datetime import datetime
import requests
from datetime import date


In [2]:
# ai_options
Normalization_File= "False"
Model_FileName= "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5"
WINDOW_SIZE= 7
PRESSISION=0.0
PERIODE_START="2023-02-09 00:00:00"
PERIODE_END="2023-02-11 00:00:00"

# start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
# end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')
start_period = pd.Timestamp(PERIODE_START)
end_period = pd.Timestamp(PERIODE_END)


#trading_options:
TAKE_PROFIT= 0.4  
STOP_LOSS=  1.8 
MAX_HOLDING_TIME= 16

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  

PAIR_WITH= "USDT"
TRADE_TOTAL= 100 
TRADE_SLOTS= 5

TRADING_FEE= 0.1
  

# --3 Importing DATA

In [ ]:
def download_coin_data(LISTCOIN, PAIR_WITH):
    coins_to_download = ''
    for coin in LISTCOIN:
        coins_to_download = coins_to_download + " " + coin
    os.system(f"node database/dd_for_backtest.js {coins_to_download} {PAIR_WITH}")

def get_volatile_pairs(halal_file="/UltimeTradingBot/Binance-Fast-Trade-Bot/halal_coins.txt"):
    url = 'https://api.binance.com/api/v3/ticker/price'
    response = requests.get(url)
    tickers = response.json()
    ticker_list = [ticker['symbol'] for ticker in tickers]
    
    with open(halal_file, "r") as f:
        VOLATILE_COINS = [line.strip() for line in f.readlines()]
    
    VOLATILE_USDT_PAIRS = [f"{coin}/USDT" for coin in VOLATILE_COINS]
    VOLATILE_BUSD_PAIRS = [f"{coin}/BUSD" for coin in VOLATILE_COINS]
    
    # Remove BUSD pairs not listed in Binance
    VOLATILE_BUSD_PAIRS = [pair for pair in VOLATILE_BUSD_PAIRS if pair.replace('/', '') in ticker_list]
    
    # Remove USDT pairs not listed in Binance
    VOLATILE_USDT_PAIRS = [pair for pair in VOLATILE_USDT_PAIRS if pair.replace('/', '') in ticker_list]
    
    # Remove USDT pairs that don't have 1m data
    content = os.listdir('database/DataBackTest/1m')
    VOLATILE_USDT_PAIRS = [pair for pair in VOLATILE_USDT_PAIRS if f"{pair.replace('/', '-')}.csv" in content]
    
    # Remove BUSD pairs that don't have 1m data
    VOLATILE_BUSD_PAIRS = [pair for pair in VOLATILE_BUSD_PAIRS if f"{pair.replace('/', '-')}.csv" in content]
    
    return VOLATILE_USDT_PAIRS + VOLATILE_BUSD_PAIRS , VOLATILE_USDT_PAIRS ,VOLATILE_BUSD_PAIRS
WORKING_PAIRS,VOLATILE_USDT_PAIRS ,VOLATILE_BUSD_PAIRS=get_volatile_pairs()

MetaData=read_metadata(WORKING_PAIRS)

In [ ]:
pair_list=VOLATILE_USDT_PAIRS
def load_historical_data(pair_list, exclude_list=[], oldest_pair="BTC/USDT"):
    if oldest_pair not in pair_list:
        pair_list.append(oldest_pair)

    df_list1m = {}
    df_list1d = {}
    df_list1h = {}
    df_list5m = {}
    df_list15m = {}

    for pair in pair_list:
        if pair in exclude_list:
            continue
        df = get_backtest_historical(ccxt.binance(), pair, '1m', path="./database/")
        df_list1m[pair] = df.loc[:]

    for pair in pair_list:
        if pair in exclude_list:
            continue
        df = get_backtest_historical(ccxt.binance(), pair, '1d', path="./database/")
        df_list1d[pair] = df.loc[:]

    for pair in pair_list:
        if pair in exclude_list:
            continue
        df = get_backtest_historical(ccxt.binance(), pair, '1h', path="./database/")
        df_list1h[pair] = df.loc[:]

    for pair in pair_list:
        if pair in exclude_list:
            continue
        df = get_backtest_historical(ccxt.binance(), pair, '5m', path="./database/")
        df_list5m[pair] = df.loc[:]

    for pair in pair_list:
        if pair in exclude_list:
            continue
        df = get_backtest_historical(ccxt.binance(), pair, '15m', path="./database/")
        df_list15m[pair] = df.loc[:]

    del(df)
    df_list = df_list1m
    print("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")
    
    return (df_list1m, df_list1d, df_list1h, df_list5m, df_list15m)



In [ ]:
def create_portfolio_dataframe(num_slots, init_quantity=500, base_currency=PAIR_WITH,start_period=start_period):
    columns = ['date', f'reserve_{base_currency}',f'total_{base_currency}']
    for i in range(num_slots):
        columns += [f'slot{i+1}_symbol', f'slot{i+1}_volume', f'slot{i+1}_original_price_{base_currency}', f'slot{i+1}_current_total_{base_currency}']
    df = pd.DataFrame(columns=columns)
    
    for date in pd.date_range(start_period-pd.Timedelta("1 minute"), periods=1):
        data = {'date': date, f'total_{base_currency}': init_quantity}
        df = df.append(data, ignore_index=True)
    
    df.set_index('date', inplace=True)
    df.iloc[:,0] = np.float64(init_quantity)
    return df.iloc[0:1]

def generate_signals(AllTop20Volumes, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, buy_test, backtest_model):
    SIGNAL_DF = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])
    for day, TOPLIST in AllTop20Volumes.items():
        for coin in TOPLIST:
            try:
                print(f">>>>>>>>>>> working on {coin} at: {day} :")
                loc_start = df_list1m[coin].index.get_loc(day)
                loc_end = df_list1m[coin].index.get_loc(day + pd.Timedelta('1 day'))
                gc.collect()
                df = mini_expand5(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData, BUY_PCT=TAKE_PROFIT, SELL_PCT=STOP_LOSS, buy_function=buy_alwase)
                dt = df.iloc[:,:-1].to_numpy(dtype=np.float32)
                predictions_note = backtest_model.predict(dt)
                predictions_round = predictions_note.round()
                dico_signal = {"coin":coin, "time":df[predictions_round==1].index.values, "price":df[predictions_round==1]["price"].values, "note":predictions_note[predictions_round==1]}
                df_signal_coin = pd.DataFrame(dico_signal)
                SIGNAL_DF = pd.concat([SIGNAL_DF, df_signal_coin])
            except:
                print(f"error at {day} in {coin}")
    return SIGNAL_DF

def init_orders_dataframe():
    return pd.DataFrame(columns=['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])

def get_top_volumes(start_period, end_period):
    top_volumes = {}
    for day, df in df_list1d["BTC/USDT"].loc[start_period:end_period].iterrows():
        top_20_vol = {}
        for p, vol_df in df_list1d.items():
            if p not in ["EUR/USDT", "EUR/BUSD", "BTC/USDT", "BTC/BUSD"]:
                try:
                    top_20_vol[p] = vol_df.loc[day].volume * vol_df.loc[day].close
                except Exception as e:
                    print(f'Time Error while working on {p}: {e}')
        top_volumes[day] = sorted(top_20_vol, key=top_20_vol.get, reverse=True)[:20]
    return top_volumes


def is_coin_in_portfolio(symbol, instant, portfolio):
    for i in range(1, TRADE_SLOTS+1):
        try:
            if portfolio[f"slot{i}_symbol"].loc[instant] == symbol:
                return i
        except Exception as e:
            print(f"Error: {e}")
    return False





In [ ]:

def get_closest_index(df, date):
    idx = df.index.searchsorted(date)
    if idx >= len(df):
        return len(df) - 1
    elif idx == 0:
        return 0
    else:
        before = df.index[idx - 1]
        after = df.index[idx]
        if after - date < date - before:
            return idx
        else:
            return idx - 1

def find_outersection(list1, list2):
    """
    Finds the outersection of two lists, i.e. the items that appear in only one of the lists.
    
    Args:
        list1 (list): The first list.
        list2 (list): The second list.
    
    Returns:
        list: The outersection of the two lists.
    """
    intersection = set(list1).intersection(set(list2))
    return list(filter(lambda x: x not in intersection, list1 + list2))

def get_top_list_in_range(all_top_20_volumes):
    top_list_in_range = []
    for day, top_list in all_top_20_volumes.items():
        top_list_in_range = top_list_in_range+find_outersection(top_list_in_range, top_list)
    return top_list_in_range


def mini_expand5(pair="GMT/USDT", i=0, j=10000, window=2, metadata=MetaData,
                 high_weight=1, BUY_PCT=BUY_PCT, SELL_PCT=SELL_PCT,
                 buy_function=buy_alwase):
    start_index=i
    end_index=j
    window_size=window
    buy_fn=buy_function    
    print(f"mini_expand : {pair}")
    pair_df = df_list1m[pair].iloc[start_index:end_index]
    btc_df = df_list1m["BTC/USDT"].loc[(pair_df.index[0] - pd.DateOffset(days=window_size+1)).round(freq='1 min'):pair_df.index[-1]+pd.Timedelta(f"{window_size} day")]
    pair_full = full_expand(pair_df, df_list5m[pair], df_list15m[pair], df_list1h[pair], df_list1d[pair], window_size)
    btc_full = full_expand(btc_df, df_list5m["BTC/USDT"], df_list15m["BTC/USDT"], df_list1h["BTC/USDT"], df_list1d["BTC/USDT"], window_size)   
    btc_full = btc_full.add_prefix("BTC_")
    merged = pd.merge(pair_full, btc_full, left_index=True, right_index=True)
    day_expand(merged)
    Meta_expand(merged, metadata, pair)
    merged = buy_fn(merged, BUY_PCT=BUY_PCT, SELL_PCT=SELL_PCT, window=MAX_FORCAST_SIZE)
    merged["high"] = (merged["open"] + high_weight * merged["high"] + merged["low"] + merged["close"]) / (3 + high_weight)
    merged["BTC_high"] = (merged["BTC_open"] + high_weight * merged["BTC_high"] + merged["BTC_low"] + merged["BTC_close"]) / (3 + high_weight)
    merged.rename(columns={"high":"price"},inplace=True)
    merged.rename(columns={"BTC_high":"BTC_price"},inplace=True)
    merged = merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    for key in merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            merged[key]=(merged["BTC_price"]-merged[key])/merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            merged[key]=(merged["price"]-merged[key])/merged["price"]
    merged=merged.dropna()
    print(f'######################  mini_expand5 {pair} - shape {merged.shape}  buy mean : {hp(merged.buy.mean())} ############################')
    return merged


def expand_data(TOP_LIST_IN_RANGE, start_period, end_period, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, Model_FileName):
    EXPENDED_DATA = {}
    backtest_model=load_model(Model_FileName)
    for coin in TOP_LIST_IN_RANGE:
        try:
            loc_start = get_closest_index(df_list1m[coin],start_period - pd.Timedelta('1 day')) 
            loc_end = get_closest_index(df_list1m[coin],end_period + pd.Timedelta('1 day')+pd.Timedelta('5 minute'))
            print(f'-------------------------- Expending {coin} from {loc_start} to {loc_end} --------------------------------')
            EXPENDED_DATA[coin] = mini_expand5(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData,
                                BUY_PCT=TAKE_PROFIT, SELL_PCT=STOP_LOSS, buy_function=buy_alwase)
            EXPENDED_DATA[coin].buy = backtest_model.predict(EXPENDED_DATA[coin].iloc[:,:-1].to_numpy(dtype=np.float32))
            EXPENDED_DATA[coin].rename(columns={"buy":"note"},inplace=True)
            EXPENDED_DATA[coin]["time"] = EXPENDED_DATA[coin].index.values
        except Exception as e:
            print(f"xxxxxxxxxxx Error: {coin} xxxxxxxxxxxxx")
            print(e)
    return EXPENDED_DATA

def generate_signal_df(AllTop20Volumes, EXPENDED_DATA):
    SIGNAL_DF = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])
    for day, TOPLIST in AllTop20Volumes.items():
        for coin in TOPLIST:
            try:
                print(f">>>>>>>>>>> working on {coin} at: {day} :")
                loc_start = get_closest_index(EXPENDED_DATA[coin],day-pd.Timedelta('1 minute'))
                loc_end = get_closest_index(EXPENDED_DATA[coin],day + pd.Timedelta('1 day')+pd.Timedelta('1 minute'))
                gc.collect()
                df = EXPENDED_DATA[coin].iloc[loc_start:loc_end,:]
                predictions_round = df['note'].round()
                #print(predictions_round)
                # Filter the rows with positive predictions and create a dataframe with the signal
                df_signal_coin = df[predictions_round == 1][['time', 'price', 'note']].reset_index(drop=True)
                df_signal_coin['coin'] = coin
                SIGNAL_DF = pd.concat([SIGNAL_DF, df_signal_coin])
            except Exception as e:
                print(f"error at {day} in {coin}")
                print(traceback.format_exc().splitlines()[-2])
                print(e)
                
    return SIGNAL_DF


In [ ]:
exclude_list = ['GBP/USDT', 'EUR/USDT', 'DNT/USDT','UST/USDT']
df_list1m, df_list1d, df_list1h, df_list5m, df_list15m=load_historical_data(pair_list, exclude_list=exclude_list)

Portfolio = create_portfolio_dataframe(TRADE_SLOTS, init_quantity=500, base_currency=PAIR_WITH)
Order_history = init_orders_dataframe()
AllTop20Volumes=get_top_volumes(start_period,end_period)

In [ ]:
Model_FileName="/UltimeTradingBot/Data/BUY_OPTIMAL/tp180_w7_max7min_Model_true_win_model_Re5.h5"
TOP_LIST_IN_RANGE=get_top_list_in_range(AllTop20Volumes)
EXPENDED_DATA = expand_data(TOP_LIST_IN_RANGE, start_period, end_period, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, Model_FileName=Model_FileName)

In [ ]:
EXPENDED_DATA["XRP/USDT"]

In [ ]:
SIGNAL_DF = generate_signal_df(AllTop20Volumes, EXPENDED_DATA)

In [ ]:
def fisrt_empty_slot(time,Portfolio=Portfolio):
    previous_min=time-pd.Timedelta("1 minute")
    for i in range(1,TRADE_SLOTS+1):
        if pd.isna(Portfolio[f'slot{i}_symbol'].loc[previous_min]):
            return i
    return False

def buy_coin(time,coin,price,Portfolio=Portfolio,pair_whith_qte=TRADE_TOTAL):
    previous_minute=time-pd.Timedelta("1 minute")
    slot_num = is_coin_in_portfolio(coin, previous_minute, Portfolio)
    previous_reserve=Portfolio.loc[previous_minute,f'reserve_{PAIR_WITH}']
    slot_ft=fisrt_empty_slot(previous_minute,Portfolio)
    if (slot_num==False) and (previous_reserve >= pair_whith_qte ) :
        Portfolio.at[time,f'reserve_{PAIR_WITH}']=previous_reserve-pair_whith_qte
        Portfolio.at[time,f'total_{PAIR_WITH}']=Portfolio.at[previous_minute,f'total_{PAIR_WITH}']-TRADING_FEE*pair_whith_qte/100
        Portfolio.at[time,f'slot{slot_ft}_symbol']=coin
        Portfolio.at[time,f'slot{slot_ft}_volume']=(pair_whith_qte-(TRADING_FEE*pair_whith_qte/100))/price
        Portfolio.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}']=price
        Portfolio.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']=price*Portfolio.at[time,f'slot{slot_ft}_volume']
        print(f"good condition for buying {Portfolio.at[time,f'slot{slot_ft}_volume']} x {coin} at {price} ")
    else:
        print(f'xxx we cannot buy  {coin} at {price}  in {time} xxx')
    return Portfolio

def sell_coin(time, coin, price, Portfolio, slot_num):
    previous_minute = time - pd.Timedelta("1 minute")
    previous_reserve = Portfolio.loc[previous_minute, f"reserve_{PAIR_WITH}"]
    previous_total = Portfolio.loc[previous_minute, f"total_{PAIR_WITH}"]
    slot_volume = Portfolio.loc[previous_minute, f"slot{slot_num}_volume"]
    slot_total_value = Portfolio.loc[previous_minute, f"slot{slot_num}_current_total_{PAIR_WITH}"]
    sold_total_value = slot_volume * price
    trade_fee = sold_total_value * TRADING_FEE / 100
    received_total_value = sold_total_value - trade_fee
    new_reserve = previous_reserve + received_total_value
    new_total = previous_total - trade_fee
    Portfolio.at[time, f"reserve_{PAIR_WITH}"] = new_reserve
    Portfolio.at[time, f"total_{PAIR_WITH}"] = new_total
    Portfolio.at[time, f"slot{slot_num}_symbol"] = np.nan
    Portfolio.at[time, f"slot{slot_num}_volume"] = np.nan
    Portfolio.at[time, f"slot{slot_num}_original_price_{PAIR_WITH}"] = np.nan
    Portfolio.at[time, f"slot{slot_num}_current_total_{PAIR_WITH}"] = np.nan
    print(f"Sold {slot_volume} x {coin} at {price} with a trade fee of {trade_fee:.6f}")
    return Portfolio


# def upadate_slots(time,df_list1m,Portfolio=Portfolio):
#     previous_minute=time-pd.Timedelta("1 minute")
#     previous_reserve=Portfolio.at[previous_minute,f'reserve_{PAIR_WITH}']
#     Portfolio.at[time,f'reserve_{PAIR_WITH}']=previous_reserve
#     previous_total=Portfolio.at[previous_minute,f'total_{PAIR_WITH}']
#     for slot_ft in range(1,TRADE_SLOTS+1):
#         if not pd.isna(Portfolio.at[previous_minute,f'slot{slot_ft}_symbol']):
#             #get previous data
#             coin=Portfolio.at[previous_minute,f'slot{slot_ft}_symbol']
#             bought_at=Portfolio.at[previous_minute, f'slot{slot_ft}_original_price_{PAIR_WITH}']
#             previous_total_slot_value=Portfolio.at[previous_minute, f'slot{slot_ft}_current_total_{PAIR_WITH}']
#             slot_volume=Portfolio.at[previous_minute,f'slot{slot_ft}_volume']
#             #Computation 
#             new_total_slot_value=df_list1m[coin].at["","close"]*slot_volume
#             slot_value_difference=new_total_slot_value-previous_total_slot_value
#             new_total_assets_value=previous_total+slot_value_difference
#             #update the slot using new price form df_list1m dataset
#             Portfolio.at[time,f'slot{slot_ft}_symbol']=coin
#             Portfolio.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}']=bought_at
#             Portfolio.at[time,f'slot{slot_ft}_volume']=slot_volume
#             Portfolio.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']=new_total_slot_value
#             Portfolio.at[time,f'total_{PAIR_WITH}']=new_total_assets_value           
#     return Portfolio
  
def stop_loss_or_take_profit(time, df_list1m, Portfolio, st_pct, tp_pct, trade_fee):
    
    for slot_ft in range(1, TRADE_SLOTS+1):
        coin = Portfolio.at[time, f"slot{slot_ft}_symbol"]
        if Portfolio.at[time, f"slot{slot_ft}_symbol"] == coin:
            # Check if stop loss or take profit order should be triggered
            bought_price = Portfolio.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"]
            slot_volume = Portfolio.at[time, f"slot{slot_ft}_volume"]
            current_price = df_list1m[coin].at[time, "close"]
            current_low_price = df_list1m[coin].at[time, "low"]
            current_high_price = (df_list1m[coin].at[time, "high"] + current_price) / 2
            slot_total_value = current_price * slot_volume
            slot_current_profit = ((current_price - bought_price) * slot_volume) - (trade_fee / 100 * slot_total_value)
            slot_current_profit_pct = (slot_current_profit / (bought_price * slot_volume)) * 100

            slot_current_low_profit = ((min(current_low_price, bought_price * (1 - st_pct / 100)) - bought_price) * slot_volume) - (trade_fee / 100 * slot_total_value)
            slot_current_low_profit_pct = (slot_current_low_profit / (bought_price * slot_volume)) * 100

            slot_current_high_profit = ((current_high_price - bought_price) * slot_volume) - (trade_fee / 100 * slot_total_value)
            slot_current_high_profit_pct = (slot_current_high_profit / (bought_price * slot_volume)) * 100

            if slot_current_profit_pct <= -st_pct:
                print(f"STOP LOSS triggered {slot_current_profit_pct}% for {coin} in slot {slot_ft} at {current_price}")
                Portfolio = sell_coin(time, coin, current_price, Portfolio, slot_ft)
                
            elif slot_current_low_profit_pct <= -st_pct:
                print(f"STOP LOSS (LOW {slot_current_low_profit_pct}%) triggered for {coin} in slot {slot_ft} at {current_low_price}")
                Portfolio = sell_coin(time, coin, current_low_price, Portfolio, slot_ft)

            elif slot_current_profit_pct >= tp_pct:
                print(f"TAKE PROFIT triggered {slot_current_profit_pct} % for {coin} in slot {slot_ft} at {current_price}")
                Portfolio = sell_coin(time, coin, current_price, Portfolio, slot_ft)

            elif slot_current_high_profit_pct >= tp_pct:
                print(f"TAKE PROFIT (HIGH {slot_current_high_profit_pct} %) triggered for {coin} in slot {slot_ft} at {current_high_price}")
                Portfolio = sell_coin(time, coin, current_high_price, Portfolio, slot_ft)

    return Portfolio

def update_slots(time, df_list1m, portfolio):
    """
    Updates the portfolio based on the current prices in df_list1m.

    Parameters:
        time (pd.Timestamp): The current time.
        df_list1m (dict): A dictionary of DataFrames containing OHLCV data for each symbol.
        portfolio (pd.DataFrame): A DataFrame containing the portfolio information.

    Returns:
        pd.DataFrame: The updated portfolio.
    """
    try:    
        pair_with = PAIR_WITH  # Assuming this is the correct value for PAIR_WITH
        trade_slots = TRADE_SLOTS  # Assuming this is the correct value for TRADE_SLOTS
        
        previous_minute = time - pd.Timedelta("1 minute")
        previous_reserve = portfolio.at[previous_minute, f"reserve_{pair_with}"]
        portfolio.at[time, f"reserve_{pair_with}"] = previous_reserve
        print(f'new reserve:{portfolio.at[time, f"reserve_{pair_with}"]}' )
        previous_total = portfolio.at[previous_minute, f"total_{pair_with}"]
        new_total_assets_value = previous_total

        
        for slot_ft in range(1, trade_slots+1):
            print(f"working to update slot number: {slot_ft}" )
            if pd.isna(portfolio.at[previous_minute, f"slot{slot_ft}_symbol"]):
                print(f"slot  {slot_ft} is empty" )
                continue  # Skip empty slots
            
            coin = portfolio.at[previous_minute, f"slot{slot_ft}_symbol"]
            bought_at = portfolio.at[previous_minute, f"slot{slot_ft}_original_price_{pair_with}"]
            previous_total_slot_value = portfolio.at[previous_minute, f"slot{slot_ft}_current_total_{pair_with}"]
            slot_volume = portfolio.at[previous_minute, f"slot{slot_ft}_volume"]
            
            try:
                new_total_slot_value = df_list1m[coin].loc[time, "close"] * slot_volume
            except Exception as e:
                print(f"Maybe No OHLCV data found for {coin}")
                continue
            
            slot_value_difference = new_total_slot_value - previous_total_slot_value
            new_total_assets_value =new_total_assets_value + slot_value_difference
            # Update the slot using new price from df_list1m dataset
            portfolio.at[time, f"slot{slot_ft}_symbol"] = coin
            portfolio.at[time, f"slot{slot_ft}_original_price_{pair_with}"] = bought_at
            portfolio.at[time, f"slot{slot_ft}_volume"] = slot_volume
            portfolio.at[time, f"slot{slot_ft}_current_total_{pair_with}"] = new_total_slot_value
            
        portfolio.at[time, f"total_{pair_with}"] = new_total_assets_value
    except Exception as e:
            print(f"what is the problem")
            print(traceback.format_exc().splitlines()[-2])
    
    return portfolio


    
import traceback        
def backtest_buy_sell(signal_df, df_list1m, start_period, end_period):
    # Define some constants
    BUY_SIDE = 'BUY'
    SELL_SIDE = 'SELL'
    EXECUTED_STATUS = 'EXECUTED'
    CANCELLED_STATUS = 'CANCELLED'
    #initialize vaiable and counters
    curent_time=pd.Timestamp(start_period)
    end_period_marge=end_period+pd.Timedelta(f"{2*MAX_HOLDING_TIME} minute")
    Portfolio=create_portfolio_dataframe(TRADE_SLOTS,init_quantity=TRADE_TOTAL* TRADE_SLOTS , base_currency=PAIR_WITH,start_period=start_period) #pair with plus slots plus total
    Order_history= pd.DataFrame(columns= ['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
    PreiodRange = pd.date_range(start=pd.Timestamp(start_period), end=pd.Timestamp(end_period), freq='1min')
    # Verifie buying oportunities
    for curent_time in PreiodRange:
        try:
            print(f"working at {curent_time}")
            previous_minute=curent_time-pd.Timedelta("1 minute")
            #previous_reserve=Portfolio.at[previous_minute,f'reserve_{PAIR_WITH}']
            try: 
                portfolio=update_slots(curent_time,df_list1m,Portfolio)
            except Exception as e:
                print("error in update the portfolio")   
                print(e)
                break     
            print(curent_time)
            for i,sig_row in signal_df[signal_df.time==pd.Timestamp(curent_time)].iterrows():
                # buy the coin if solt and cash is avalible:
                coin = sig_row['coin']
                time = sig_row['time']
                price = sig_row['price'] # We assume there's only one price per signal
                note = sig_row['note'] 
                slot_num = is_coin_in_portfolio(coin, previous_minute, Portfolio)
                # buy first:
                if (note + PRESSISION) >= 0.5:
                    Portfolio=buy_coin(time,coin,price,Portfolio=Portfolio,pair_whith_qte=TRADE_TOTAL)
            
            Portfolio=stop_loss_or_take_profit(curent_time, df_list1m, Portfolio, STOP_LOSS , TAKE_PROFIT, TRADING_FEE )
            
            ptc_benifit=100*Portfolio.at[curent_time,f'total_{PAIR_WITH}']/Portfolio[f'total_{PAIR_WITH}'].iloc[0]
            print(f"total benifit ptc {ptc_benifit}")       
        except Exception as e:
            print(f"xxxxxxxxxxxxxxxxxxxxxx Errot at {curent_time} xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
            print(traceback.format_exc().splitlines()[-2])
            print(e)
            print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    return Portfolio     
                
                
            
        
            

In [ ]:
MetaData

In [ ]:
Portfolio=backtest_buy_sell(SIGNAL_DF, df_list1m, start_period, end_period=pd.Timestamp("2023-02-09 00:10:00"))

In [ ]:
SIGNAL_DF

In [ ]:
SIGNAL_DF[SIGNAL_DF["time"]==pd.Timestamp("2023-02-09 00:01:00")]

In [ ]:
Portfolio

# Binance coins Info

In [41]:
import requests
import pandas as pd

def get_binance_coin_list1():
    api_endpoint = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(api_endpoint)
    binance_coins = [symbol["baseAsset"] for symbol in response.json()["symbols"]]
    return binance_coins


def get_binance_coin_list():
    api_endpoint = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(api_endpoint)
    binance_coins = [symbol["baseAsset"].lower() for symbol in response.json()["symbols"]]
    return binance_coins

# rest of the code remains the same

def get_coingecko_coin_list1():
    api_endpoint = "https://api.coingecko.com/api/v3/coins/list"
    response = requests.get(api_endpoint)
    coingecko_coins = [coin["id"] for coin in response.json()]
    return coingecko_coins

import requests
import json
import pandas as pd

def get_coingecko_coin_list():
    api_endpoint = "https://api.coingecko.com/api/v3/coins/list"
    response = requests.get(api_endpoint)
    response_json = json.loads(response.text)
    coingecko_coins = [coin["id"] for coin in response_json]
    return coingecko_coins

# rest of the code remains the same


def get_coin_data(coin):
    api_endpoint = f"https://api.coingecko.com/api/v3/coins/{coin}"
    response = requests.get(api_endpoint)
    data = response.json()

    website = data["links"]["homepage"][0] if len(data["links"]["homepage"]) > 0 else None
    twitter = data["links"]["twitter_screen_name"] if "twitter_screen_name" in data["links"] else None
    explorer = data["links"]["blockchain_site"][0] if len(data["links"]["blockchain_site"]) > 0 else None
    full_description = data["description"]["en"]
    category = data["categories"][0] if len(data["categories"]) > 0 else None

    return {"coin": coin, "website": website, "twitter": twitter, "explorer": explorer, "full description": full_description, "category": category}

def get_coins_data():
    binance_coins = get_binance_coin_list()
    # coingecko_coins = get_coingecko_coin_list()
    common_coins = binance_coins #list(set(binance_coins) & set(coingecko_coins))

    coins_data = []
    for coin in common_coins:
        coin_data = get_coin_data(coin)
        coins_data.append(coin_data)

    df = pd.DataFrame(coins_data, columns=["coin", "website", "twitter", "explorer", "full description", "category"])
    return df


In [51]:
binance_coins = get_binance_coin_list()
# coingecko_coins = get_coingecko_coin_list1()
common_coins = list(set(binance_coins) & set(coingecko_coins))

# coins_data = []
# for coin in common_coins:
#     coin_data = get_coin_data(coin)
#     coins_data.append(coin_data)

# df = pd.DataFrame(coins_data, columns=["coin", "website", "twitter", "explorer", "full description", "category"])
# return df

In [47]:
api_endpoint = "https://api.coingecko.com/api/v3/coins/list"
response = requests.get(api_endpoint)
response_json = json.loads(response.text)

In [49]:
response

<Response [429]>